In [1]:
import sys 
sys.path.append("../../../")
from simulation_classes.eta.PathPlanner import PathPlanner

In [6]:
# initial_params
initial_time =20
v_0=20
initial_params = {"time": initial_time, "speed":v_0}

# ideal_params
v_exit = 30
ideal_arrival_time = 60
ideal_params_at_end = {"ideal_arrive_time": ideal_arrival_time, "ideal_speed": v_exit}

# car_spec
max_acc = 2
max_dec = 2
v_max = 45
car_spec = {"max_acc":max_acc, "max_dec":max_dec, "v_max":v_max}

#その他の設計
L_max = 500

pathPlanner = PathPlanner(car_spec = car_spec, initial_params = initial_params, 
                          ideal_params_at_end=ideal_params_at_end, COURSE_LENGTH = L_max)

x = pathPlanner.solve_path(priority="speed")
print(x)

{'is_possible': True, 'Reason': 'OK'}
PROFILE:  DCA

m1=10.00, m2=25.00, m3=5.00
D1=200.00, D2=250.00, D3=75.00

m1=12.50, m2=20.00, m3=7.50
D1=218.75, D2=100.00, D3=93.75

m1=11.25, m2=22.50, m3=6.25
D1=210.94, D2=168.75, D3=85.94

m1=10.62, m2=23.75, m3=5.62
D1=205.86, D2=207.81, D3=80.86

m1=10.31, m2=24.38, m3=5.31
D1=203.03, D2=228.52, D3=78.03

m1=10.47, m2=24.06, m3=5.47
D1=204.47, D2=218.07, D3=79.47

m1=10.55, m2=23.91, m3=5.55
D1=205.17, D2=212.92, D3=80.17

m1=10.51, m2=23.98, m3=5.51
D1=204.82, D2=215.48, D3=79.82

m1=10.53, m2=23.95, m3=5.53
D1=205.00, D2=214.20, D3=80.00
9
m1の解=10.52

m1=10.52, m2=23.96, m3=5.52
D1=204.91, D2=214.84, D3=79.91
距離:499.66
[{'ACC': -2, 'duration': 5.517578125, 'initial_speed': 20}, {'ACC': 0, 'duration': 23.96484375, 'initial_speed': 8.96484375}, {'ACC': 2, 'duration': 10.517578125, 'initial_speed': 8.96484375}]
None


In [151]:
"""
クラスを書き直したことでこれは不要になった
"""

def calc_controlled_eta(initial_params, ideal_params_at_end, car_spec, L_max ):
    v_0 = initial_params["speed"]
    v_lim = car_spec["v_max"]
    v_exit = ideal_params_at_end["ideal_speed"]
    a_max = car_spec["max_acc"]
    a_dec = car_spec["max_dec"]
    t_0 = initial_params["time"]
    t_end = ideal_params_at_end["ideal_arrive_time"]
    L_max = L_max

    if v_0 > v_lim or v_exit > v_lim:
        ValueError("Enter Speedが最高速度を上回っています")

    if v_0 > v_exit:
        ValueError("Enter Speedが出口速度を上回っています")

    if (t_end - t_0) * a_max + v_0 < v_exit:
        print("不可能: 全力で加速するのみ")
        return False

    if (t_end - t_0) * v_0 >= L_max: # Trueだった場合、早く着きすぎているということ
        print("Possible: 早く着きすぎた")
        return True
        
    # ここより上の議論は(v_0, v_exit) と (t_0, t_end) だけで完結する話

    
    # 以下、距離まで織り込んだ議論 
    # 終点で速度をv_exitにするという条件のもとで頑張る
    t_margin = t_end - t_0
    time_to_reach_v_exit = (v_exit - v_0) / a_max
    v_max_possible = (t_margin - time_to_reach_v_exit) / (a_max + a_dec) * a_dec * a_max + v_exit

    print("可能的最高速度:", v_max_possible)

    if v_max_possible > v_lim:
        # この場合は一定速度 v_lim で走る区間がある、s1+s2+s3 > LならOKのやつ
        print("a")
        s_1 = (v_lim**2 - v_0 ** 2) / 2 / a_max
        s_3 = (v_lim**2 - v_exit ** 2) / 2 / a_dec
        t_1 = (v_lim - v_0) / a_max
        t_3 = (v_lim - v_exit) / a_dec
        t_2 = t_end - t_0 - t_1 - t_3
        S = s_1 + s_3 + t_2 * v_lim

        print("可能的最高速度:", v_lim)
        print("t_2=",t_2)
        print("t_3=",t_3)
        print("カバー距離:", S)
        print("達成可能性:", (t_2 < 0 or S > L_max))

        return (t_2 < 0 or S > L_max)

    # これより下は v_lim <= v_maxの場合。要するに行けるとこまで加速する
    # v_0で入ってからv_exitで抜けるまでの総走行距離を計算する
    print("定速区間なし")
    
    S = (v_max**2 - v_0**2) / 2 / a_max + (v_max**2 - v_0**2) / 2 / a_dec
    total_time = (v_max - v_0)/a_max + (v_max - v_exit) / a_dec
    can_cover_the_distance = S > L_max
    is_in_time = total_time < t_margin
    print("カバー距離:", S)
    print(can_cover_the_distance, is_in_time)
    print("達成可能性:", ( is_in_time and can_cover_the_distance))

    return (is_in_time and can_cover_distance)
    